In [41]:
import pandas as pd
import numpy as np

In [49]:
#load historical mean of data for month we can't predict (we cant predict EVI data, we cant predict weather for 9 month in future etc) so we remplace prediction with mean historical values
hist_temp = pd.read_csv("temp_precip_mean_historical.csv")
hist_evi = pd.read_csv("evi_mean_historical.csv")
hist_vpd = pd.read_csv("vpd_mean_historical.csv")

#load current market year data, without data from current month
current_weather_vpd = pd.read_csv("temp_precip_vpd_mean_current.csv")
current_evi = pd.read_csv("evi_mean_current.csv")

#load forecast data
forecast_weather_vpd = pd.read_csv("weather_forecast.csv")

In [ ]:
hist_evi_month = np.setxor1d(hist_evi["MONTH"].unique(), current_evi["MONTH"].unique()) #get month that we do not have in current data 
hist_evi = hist_evi[hist_evi["MONTH"].isin(hist_evi_month)] #only keep month were we do not have the current data

forecast_weather_vpd_month = np.setxor1d(hist_temp["MONTH"].unique(), current_weather_vpd["MONTH"].unique()) #same for forecast data
forecast_weather_vpd = forecast_weather_vpd[forecast_weather_vpd["MONTH"].isin(forecast_weather_vpd_month)]